In [1]:
import os
import sys
import numpy as np
import copy
import scipy
import pywt

from sklearn.preprocessing import normalize

sys.path.insert(0, "/home/anonymous/Desktop/eeg_sampling/modeling")
from dataio import readdata2, readlabels2, writedata
from datafilters import apply_dc_filter

In [2]:
def apply_dwt_filter(y, dwt_type, dwt_level, dwt_thresh_func, dwt_thresh_type):
    coeffs = pywt.wavedecn(y, dwt_type, level=dwt_level)
    for i in range(1, dwt_level+1):
        coeffs[i]["d"] = pywt.threshold(coeffs[i]["d"], thselect(coeffs[i]["d"], dwt_thresh_type), dwt_thresh_func)
    return(pywt.waverecn(coeffs, dwt_type))

In [3]:
dataset = readdata2("./../curated/raw-samples")
labels = readlabels2("./../curated/raw-inputs")

#Constants
fs = 250.0 #Frequency in Hz
sample_time = dataset[0].shape[1]/fs #Total time for sample
num_rows = 360 
num_channels = 8
rto = num_rows/(fs/2)

In [4]:
master_dataset = []

# Channel 1 (For right hand movement)
data = []
for i in range(0, len(dataset)):
    data.append(np.delete(dataset[i], [0,2,3,4,5,6,7], 2))
    data[i].shape = (data[i].shape[0], data[i].shape[1])
master_dataset.append(data)

# Channel 3 (For left hand movement)
data = []
for i in range(0, len(dataset)):
    data.append(np.delete(dataset[i], [0,1,2,4,5,6,7], 2))
    data[i].shape = (data[i].shape[0], data[i].shape[1])
master_dataset.append(data)
  
# Grand Average
data = []
for i in range(0,len(dataset)):
    data.append(np.mean(dataset[i],axis=2))
master_dataset.append(data)

In [5]:
#First filter
dc_lowcut = 1.0
dc_highcut = 13.0
dc_order = 2
dc_type = "bandpass"
dc_func_type = "butter"
for i in range(0, len(master_dataset)):
    for j in range(0, len(master_dataset[i])):
        for k in range(0, master_dataset[i][j].shape[0]):
            master_dataset[i][j][k] = apply_dc_filter(master_dataset[i][j][k], fs, dc_lowcut, dc_highcut, dc_order, dc_type, dc_func_type)

In [6]:
def data_summary(x):
    return(x.max(), x.min(), x.mean(), x.std(), x.var())

In [7]:
#Discrete Wavelet Transform
dwt_type = "db2"
dwt_level = 4
dwt_thresh_func = "soft"
dwt_thresh_type = "rigrsure"

master_dwt = []
for i in range(0, len(master_dataset)):
    trial = []
    for j in range(0, len(master_dataset[i])):
        sample = []
        for k in range(0, master_dataset[i][j].shape[0]):
            dwt_tmp = pywt.wavedecn(master_dataset[i][j][k], dwt_type, level=dwt_level) #Compute DWT\
            summ = []
            summ.append(data_summary(dwt_tmp[0]))
            for m in range(1, dwt_level+1):
                summ.append(data_summary(dwt_tmp[m]["d"]))
            summ = np.array(summ)
            sample.append(summ.flatten())
            #sample.append(dwt_tmp[4]["d"])
        trial.append(sample)
    master_dwt.append(trial)

In [ ]:
len(master_dwt[0][0])

In [8]:
# Test Set
test_index = 15 #Last one
test_dataset = []
train_dataset = []
for i in range(0, len(master_dwt)):
    test_dataset.append(master_dwt[i][test_index])
    master_dwt[i] = np.delete(master_dwt[i], test_index, axis = 0)
    train_dataset.append(master_dwt[i])
    
test_labels = labels[test_index]
labels = np.delete(labels, test_index, axis = 0)
train_labels = labels


# Train Set
for i in range(0, len(train_dataset)):
    train_dataset[i] = np.concatenate(train_dataset[i])

train_labels = np.concatenate(labels)

In [9]:
train_right = train_dataset[0].astype(np.float32)
train_left = train_dataset[1].astype(np.float32)
train_avg = train_dataset[2].astype(np.float32)
train_right.shape = (train_right.shape[0],train_right.shape[1],1)
train_left.shape = (train_left.shape[0],train_left.shape[1],1)
train_avg.shape = (train_avg.shape[0],train_avg.shape[1],1)

test_right = np.array(test_dataset[0], dtype=np.float32)
test_left = np.array(test_dataset[1], dtype=np.float32)
test_avg = np.array(test_dataset[2], dtype=np.float32)
test_right.shape = (test_right.shape[0],test_right.shape[1],1)
test_left.shape = (test_left.shape[0],test_left.shape[1],1)
test_avg.shape = (test_avg.shape[0],test_avg.shape[1],1)


#Write training
writedata("./../curated/train_dataset_right", train_right)
writedata("./../curated/train_dataset_left", train_left)
writedata("./../curated/train_dataset_avg", train_avg)
writedata("./../curated/train_labels", train_labels.astype(np.uint8))

#Write test
writedata("./../curated/test_dataset_right", test_right)
writedata("./../curated/test_dataset_left", test_left)
writedata("./../curated/test_dataset_avg", test_avg)
writedata("./../curated/test_labels", test_labels.astype(np.int8))

In [ ]:
train_dataset[0].shape